In [61]:
def applyP(v, p):
    return list(v[p[i] - 1] for i in range(0, len(p)))

def shl(v, pos_num):
    pos_num = pos_num % len(v)
    pos_num = (pos_num + len(v)) % len(v)
    res = list(v[i] for i in range(pos_num, len(v)))
    res.extend(list(v[i] for i in range(0, pos_num)))
    return res

def shr(v, pos_num):
    pos_num = pos_num % len(v)
    pos_num = (pos_num + len(v)) % len(v)
    res = list(v[i] for i in range(len(v) - pos_num, len(v)))
    res.extend(list(v[i] for i in range(0, len(v) - pos_num)))
    return res

def genRoundKeys(basicKey):
    pkey = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
    permuted_key = applyP(basicKey, pkey)
    #print('permuted_key')
    #print(permuted_key)
    round_keys = []
    left = list(permuted_key[i] for i in range(0, 5))
    right = list(permuted_key[i] for i in range(5, 10))
    #print('left')
    #print(left)
    #print('right')
    #print(right)
    p_zip = [6, 3, 7, 4, 8, 5, 10, 9]
    left = list(shl(left, 1))
    right = list(shl(right, 1))
    #print('left')
    #print(left)
    #print('right')
    #print(right)
    key1 = list(left)
    key1.extend(right)
    round_keys.append(applyP(key1, p_zip))
    left = shl(left, 2)
    right = shl(right, 2)
    #print('left')
    #print(left)
    #print('right')
    #print(right)
    key2 = left
    key2.extend(right)
    round_keys.append(applyP(key2, p_zip))
    return round_keys

In [76]:
def f_sdes(left, right, key):
    print('Entered f_sdes')
    extend_perm = [4, 1, 2, 3, 2, 3, 4, 1]
    #applying permutation for extention
    new_mes = applyP(right, extend_perm)
    print('new_mes')
    print(new_mes)
    print('left')
    print(left)
    print('right')
    print(right)
    print('key')
    print(key)
    #applying key
    keyd_mes = list(new_mes[i]^key[i] for i in range(0, len(key)))
    print('keyd_mes')
    print(keyd_mes)
    #TODO:
    #incorrect value in S0[3][3] and in S1[0][0] and in S0[2][3]
    S0 = [
        [[0, 1], [0, 0], [1, 1], [1, 0]],
        [[1, 1], [1, 0], [0, 1], [0, 0]],
        [[0, 0], [1, 0], [0, 1], [1, 1]],
        [[1, 1], [0, 1], [1, 1], [1, 0]]
    ];
    S1 = [
        [[0, 0], [0, 1], [1, 0], [1, 1]],
        [[1, 0], [0, 0], [0, 1], [1, 1]],
        [[1, 1], [0, 0], [1, 0], [0, 0]],
        [[1, 0], [1, 0], [0, 0], [1, 1]]
    ];
    after_sq_blocks = list(S0[keyd_mes[0]*2+keyd_mes[3]][keyd_mes[1]*2+keyd_mes[2]])
    after_sq_blocks.extend(S1[keyd_mes[4 + 0]*2+keyd_mes[4 + 3]][keyd_mes[4 + 1]*2+keyd_mes[4 + 2]])
    print('after_sq_blocks')
    print(after_sq_blocks)
    P4 = [2, 4, 3, 1]
    after_perm = applyP(after_sq_blocks, P4)
    print('after_perm')
    print(after_perm)
    print('Left f_sdes')
    return after_perm

In [70]:
def sdes_round(message, key):
    left = message[:4]
    right = message[4:]
    print('left')
    print(left)
    print('right')
    print(right)
    after_f = f_sdes(left, right, key)
    #print('right')
    #print(right)
    after_xor = list(left[i]^after_f[i] for i in range(0, 4))
    print('after_xor')
    print(after_xor)
    print('keyd_mes')
    #new_mes = list(right)
    #new_mes.extend(after_xor)
    new_mes = list(after_xor)
    new_mes.extend(right)
    return new_mes

In [64]:
def sdes_ready_keys(message, keys):
    start_p = [2, 6, 3, 1, 4, 8, 5, 7]
    end_p = [4, 1, 3, 5, 7, 2, 8, 6]
    mixed_mes = applyP(message, start_p)
    print('mixed_mes')
    print(mixed_mes)
    round1 = sdes_round(mixed_mes, keys[0])
    print('round1')
    print(round1)
    message_for_r2 = list(round1[4:8])
    message_for_r2.extend(round1[0:4])
    print('message_for_r2')
    print(message_for_r2)
    round2 = sdes_round(message_for_r2, keys[1])
    print('round2')
    print(round2)
    result = applyP(round2, end_p)
    print('result')
    print(result)
    return result

In [65]:
def encrypt(message, key):
    keys = genRoundKeys(key)
    print('keys')
    print(keys)
    return sdes_ready_keys(message, keys)

In [66]:
def decrypt(message, key):
    keys = genRoundKeys(key)
    keys.reverse()
    return sdes_ready_keys(message, keys)

In [44]:
genRoundKeys([0, 1, 1, 1, 1, 1, 1, 1, 0, 1])

[[0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0]]

In [7]:
applyP([0, 1, 2, 3], [2, 3, 1, 4])

[1, 2, 0, 3]

In [74]:
sdes_round([0, 0, 1, 1, 0, 0, 0, 1], [0, 1, 0, 1, 1, 1, 1, 1])

left
[0, 0, 1, 1]
right
[0, 0, 0, 1]
Entered f_sdes
new_mes
[1, 0, 0, 0, 0, 0, 1, 0]
keyd_mes
[1, 1, 0, 1, 1, 1, 0, 1]
after_sq_blocks
[1, 1, 0, 0]
after_perm
[1, 0, 0, 1]
Left f_sdes
after_xor
[1, 0, 1, 0]
keyd_mes


[1, 0, 1, 0, 0, 0, 0, 1]

In [80]:
#intuit example:
#key: [1, 0, 1, 1, 1, 0, 0, 1, 1, 0]
#message: [1, 1, 1, 1, 0, 0, 1, 0]
#encrypted text: [1, 1, 1, 0, 1, 0, 1, 1]
#example from university (cryptoprotocols discipline) lection:
#key: [0, 1, 1, 1, 1, 1, 1, 1, 0, 1]
#message: [1, 0, 1, 0, 0, 0, 1, 0]
#encrypted text: [0, 0, 1, 1, 1, 0, 0, 0]
encrypt([1, 1, 1, 1, 0, 0, 1, 0], [1, 0, 1, 1, 1, 0, 0, 1, 1, 0])

keys
[[1, 0, 1, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0, 1, 1]]
mixed_mes
[1, 0, 1, 1, 1, 0, 0, 1]
left
[1, 0, 1, 1]
right
[1, 0, 0, 1]
Entered f_sdes
new_mes
[1, 1, 0, 0, 0, 0, 1, 1]
left
[1, 0, 1, 1]
right
[1, 0, 0, 1]
key
[1, 0, 1, 1, 1, 1, 0, 0]
keyd_mes
[0, 1, 1, 1, 1, 1, 1, 1]
after_sq_blocks
[0, 0, 1, 1]
after_perm
[0, 1, 1, 0]
Left f_sdes
after_xor
[1, 1, 0, 1]
keyd_mes
round1
[1, 1, 0, 1, 1, 0, 0, 1]
message_for_r2
[1, 0, 0, 1, 1, 1, 0, 1]
left
[1, 0, 0, 1]
right
[1, 1, 0, 1]
Entered f_sdes
new_mes
[1, 1, 1, 0, 1, 0, 1, 1]
left
[1, 0, 0, 1]
right
[1, 1, 0, 1]
key
[1, 1, 0, 1, 0, 0, 1, 1]
keyd_mes
[0, 0, 1, 1, 1, 0, 0, 0]
after_sq_blocks
[1, 0, 1, 1]
after_perm
[0, 1, 1, 1]
Left f_sdes
after_xor
[1, 1, 1, 0]
keyd_mes
round2
[1, 1, 1, 0, 1, 1, 0, 1]
result
[0, 1, 1, 1, 0, 1, 1, 1]


[0, 1, 1, 1, 0, 1, 1, 1]

In [81]:
decrypt([0, 1, 1, 1, 0, 1, 1, 1], [1, 0, 1, 1, 1, 0, 0, 1, 1, 0])

mixed_mes
[1, 1, 1, 0, 1, 1, 0, 1]
left
[1, 1, 1, 0]
right
[1, 1, 0, 1]
Entered f_sdes
new_mes
[1, 1, 1, 0, 1, 0, 1, 1]
left
[1, 1, 1, 0]
right
[1, 1, 0, 1]
key
[1, 1, 0, 1, 0, 0, 1, 1]
keyd_mes
[0, 0, 1, 1, 1, 0, 0, 0]
after_sq_blocks
[1, 0, 1, 1]
after_perm
[0, 1, 1, 1]
Left f_sdes
after_xor
[1, 0, 0, 1]
keyd_mes
round1
[1, 0, 0, 1, 1, 1, 0, 1]
message_for_r2
[1, 1, 0, 1, 1, 0, 0, 1]
left
[1, 1, 0, 1]
right
[1, 0, 0, 1]
Entered f_sdes
new_mes
[1, 1, 0, 0, 0, 0, 1, 1]
left
[1, 1, 0, 1]
right
[1, 0, 0, 1]
key
[1, 0, 1, 1, 1, 1, 0, 0]
keyd_mes
[0, 1, 1, 1, 1, 1, 1, 1]
after_sq_blocks
[0, 0, 1, 1]
after_perm
[0, 1, 1, 0]
Left f_sdes
after_xor
[1, 0, 1, 1]
keyd_mes
round2
[1, 0, 1, 1, 1, 0, 0, 1]
result
[1, 1, 1, 1, 0, 0, 1, 0]


[1, 1, 1, 1, 0, 0, 1, 0]